In [1]:
import networkx as nx
from fa2 import ForceAtlas2  #the package should be installed before
import matplotlib.pyplot as plt
from datetime import datetime
from statistics import mean, median, quantiles
from networkx.algorithms.traversal.breadth_first_search import descendants_at_distance
import time
import requests
import glob
import pickle
import numpy as np
import copy
import pandas as pd
import sklearn.preprocessing

In [2]:
with open('stat_puppies_distrEqualized.pickle', 'rb') as handle:
    puppies = pickle.load(handle)
    
with open('stat_PoliticalDiscussion_distrEqualized.pickle', 'rb') as handle:
    PoliticalDiscussion = pickle.load(handle)

In [3]:
df = pd.DataFrame()

mini=10
maxi=29

ids = [id.split("_")[1] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
ids.extend([id.split("_")[1] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

subreddit = ["puppies" for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
subreddit.extend(["PoliticalDiscussion" for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

num_comments = [puppies[id]["commStats"]["num_comments"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
num_comments.extend([PoliticalDiscussion[id]["commStats"]["num_comments"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

num_direct_comments = [puppies[id]["commStats"]["num_direct_comments"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
num_direct_comments.extend([PoliticalDiscussion[id]["commStats"]["num_direct_comments"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

max_depth = [puppies[id]["commStats"]["max_depth"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
max_depth.extend([PoliticalDiscussion[id]["commStats"]["max_depth"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

H_index = [puppies[id]["commStats"]["H_index"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
H_index.extend([PoliticalDiscussion[id]["commStats"]["H_index"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

lifeTime = [puppies[id]["timeStats"]["lifeTime"]/86400 for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
lifeTime.extend([PoliticalDiscussion[id]["timeStats"]["lifeTime"]/86400 for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

averageTime = [puppies[id]["timeStats"]["averageTime"]/86400 for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
averageTime.extend([PoliticalDiscussion[id]["timeStats"]["averageTime"]/86400 for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

medianTime = [puppies[id]["timeStats"]["medianTime"]/86400 for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
medianTime.extend([PoliticalDiscussion[id]["timeStats"]["medianTime"]/86400 for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

lifetime95 = [puppies[id]["timeStats"]["lifetime95"]/86400 for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
lifetime95.extend([PoliticalDiscussion[id]["timeStats"]["lifetime95"]/86400 for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

num_users = [puppies[id]["authorStats"]["num_users"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
num_users.extend([PoliticalDiscussion[id]["authorStats"]["num_users"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

num_edges = [puppies[id]["authorStats"]["num_edges"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
num_edges.extend([PoliticalDiscussion[id]["authorStats"]["num_edges"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

size_largest_conn_comp = [puppies[id]["authorStats"]["size_largest_conn_comp"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
size_largest_conn_comp.extend([PoliticalDiscussion[id]["authorStats"]["size_largest_conn_comp"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

recip_value = [puppies[id]["authorStats"]["recip_value"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
recip_value.extend([PoliticalDiscussion[id]["authorStats"]["recip_value"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

diam = [puppies[id]["authorStats"]["diam"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
diam.extend([PoliticalDiscussion[id]["authorStats"]["diam"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])

clust_coeff = [puppies[id]["authorStats"]["clust_coeff"] for id in puppies if (puppies[id]["commStats"]["num_comments"]>mini and puppies[id]["commStats"]["num_comments"]<maxi)]
clust_coeff.extend([PoliticalDiscussion[id]["authorStats"]["clust_coeff"] for id in PoliticalDiscussion if (PoliticalDiscussion[id]["commStats"]["num_comments"]>mini and PoliticalDiscussion[id]["commStats"]["num_comments"]<maxi)])
             
df["group"] = subreddit
df["name"]=ids
df["num_comments"]=num_comments
df["num_direct_comments"]=num_direct_comments
df["max_depth"]=max_depth
df["H_index"]=H_index
df["lifeTime"]=lifeTime
df["averageTime"]=averageTime
df["medianTime"]=medianTime
df["lifetime95"]=lifetime95
df["num_users"]=num_users
df["num_edges"]=num_edges
df["size_largest_conn_comp"]=size_largest_conn_comp
df["recip_value"]=recip_value
df["diam"]=diam
df["clust_coeff"]=clust_coeff

In [4]:
#df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df

,group,name,num_comments,num_direct_comments,max_depth,H_index,lifeTime,averageTime,medianTime,lifetime95,num_users,num_edges,size_largest_conn_comp,recip_value,diam,clust_coeff
0,puppies,ccefbd,23.0,17.0,5.0,3.0,9.036424,0.974425,0.194097,3.185391,18.0,21.0,1.0,0.380952,0.0,0.000000
1,puppies,l8kw8z,16.0,13.0,4.0,2.0,17.245012,2.546542,0.452766,16.838501,14.0,14.0,1.0,0.142857,0.0,0.000000
2,puppies,cqfwfd,12.0,6.0,3.0,3.0,0.633206,0.354694,0.429410,0.626588,7.0,11.0,1.0,0.909091,0.0,0.000000
3,puppies,k0cac1,11.0,5.0,5.0,3.0,0.520845,0.170488,0.086250,0.492778,6.0,7.0,1.0,0.571429,0.0,0.000000
4,puppies,iqcrx3,20.0,11.0,6.0,3.0,1.778738,0.264876,0.140475,0.680336,16.0,17.0,3.0,0.117647,2.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,PoliticalDiscussion,i04gue,26.0,5.0,6.0,5.0,5.074178,2.597278,2.055029,4.453633,16.0,23.0,13.0,0.260870,6.0,0.135119
680,PoliticalDiscussion,f3ywpe,20.0,10.0,6.0,3.0,3.212130,2.145745,2.182963,3.116017,16.0,18.0,6.0,0.222222,3.0,0.022222
681,PoliticalDiscussion,k6a9h1,17.0,8.0,8.0,3.0,2.744653,0.602279,0.585556,1.291829,15.0,16.0,7.0,0.250000,3.0,0.000000
682,PoliticalDiscussion,ntpooj,18.0,5.0,7.0,3.0,3.036620,0.577469,0.448432,1.216172,12.0,16.0,6.0,0.500000,3.0,0.111111


In [6]:
for i, column in enumerate(df.columns):
    df_feat = df[["group","name"]]
    df_feat["value"] = df[column]
    print(i, " ",column)
    df_feat.to_csv("d3-bubble-master/HistCSV/"+column+".csv", index=False)

0   group
1   name
2   num_comments
3   num_direct_comments
4   max_depth
5   H_index
6   lifeTime
7   averageTime
8   medianTime
9   lifetime95
10   num_users
11   num_edges
12   size_largest_conn_comp
13   recip_value
14   diam
15   clust_coeff


<ipython-input-6-07d7e2a01511>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feat["value"] = df[column]


In [11]:
df.to_csv("AllStatistics.csv", index=False)

In [16]:
df_feat

,group,name,value
0,puppies,t3_ccefbd,0.000000
1,puppies,t3_l8kw8z,0.000000
2,puppies,t3_cqfwfd,0.000000
3,puppies,t3_k0cac1,0.000000
4,puppies,t3_iqcrx3,0.000000
...,...,...,...
679,PoliticalDiscussion,t3_i04gue,0.135119
680,PoliticalDiscussion,t3_f3ywpe,0.022222
681,PoliticalDiscussion,t3_k6a9h1,0.000000
682,PoliticalDiscussion,t3_ntpooj,0.111111
